# Utilisation Label Studio pour création d'un jeu de données d'évaluation de retrieval

On souhaite utiliser Label Studio sur le SSP Cloud pour créer un jeu d'évaluation de la manière suivante:

- On commence par sélectionner un échantillon de pages sur le site Insee;
- On veut une tâche d'annotation par page;
- Pour une tâche d'annotation, on affiche tout le texte de la page qui a été embeddé et stocké dans la vector database de RAG, ainsi qu'un lien vers la page du site Insee;
- Une tâche d'annotation consiste à poser une ou plusieurs questions sur le contenu qui figure dans la page;
- Cette tâche est plus facile à réaliser en lisant la page sur le site Insee, pour des raisons de mise en page. Ainsi, il faut avant de valider les questions valider que le contenu  correspondant figure bien dans le texte affiché dans Label Studio (ce texte est en général une sous-inclusion du texte figurant véritablement sur la page web).

On utilise l'interface entre Label Studio et s3 - il faut sauvegarder les tâches d'annotation sous forme de JSON.

In [1]:
import s3fs
import json

fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'}
)

In [ ]:
import pandas as pd

# Load data
with fs.open('projet-llm-insee-open-data/data/eval_data/eval_retrieval/insee_documents_sample_ref_retrieval_evaluation.csv', "r") as f:
    df = pd.read_csv(f)
df["text"] = df["content"]

# Save each annotation task as a json file
for i, row in enumerate(df[["text", "source", "title"]].iterrows()):
    json_data = {'text': row[1]['text'], 'source': row[1]['source'], 'title': row[1]['title']}
    with fs.open(f'projet-llm-insee-open-data/data/eval_data/eval_retrieval/test_tasks/{i}.json', 'w', encoding='utf8') as fp:
        json.dump(json_data, fp, ensure_ascii=False)

Un template d'annotation possible est le suivant, à renseigner sur Label Studio:

```html
<View>
  <Header value="Titre du texte"/>
  <Text name="title" value="$title"/>
  <Header value="Page source"/>
  <HyperText name="p1" clickableLinks="true" inline="true" target="_blank">
    <a target="_blank" href="$source">$source</a>
  </HyperText>
  
  <Header value="Posez une ou plusieurs questions sur le texte, en les séparant avec un pipe |. Vérifiez que le contenu correspondant à la question figure bien dans le texte suivant, qui est une extraction en général incomplète de la page."/>
  <TextArea name="answer" toName="text" showSubmitButton="true" maxSubmissions="1" editable="true" required="true"/>

  <Header value="Extraction"/>
  <Text name="text" value="$text"/>
</View>
```

L'interface avec s3 se configure dans Settings > Cloud Storage. Le code suivant permet de récupérer les annotations créées avec ce projet (https://projet-llm-insee-open-data-label-studio.user.lab.sspcloud.fr/projects/2/) et de les formatter.

In [12]:
evaluation_data = []

In [13]:
for path in fs.listdir("projet-llm-insee-open-data/data/eval_data/eval_retrieval/test_annotations/"):
    with fs.open(path["Key"], 'rb') as f:
        annotation_data = json.load(f)
        source = annotation_data["task"]["data"]["source"]
        questions = annotation_data["result"][0]["value"]["text"][0]
        questions = questions.split("|")
        questions = [q.strip() for q in questions]
        evaluation_data = evaluation_data + [{"source": source, "questions": questions}]

In [14]:
evaluation_data

[{'source': 'https://www.insee.fr/fr/information/2407785',
  'questions': ["Quel est le taux d'entreprises privées couvert par les DADS en 2017 ?",
   "Quel est le taux d'entreprises privées couvertes en 2018 ?"]}]